In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


import os
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression, Ridge, Lasso 
from sklearn.model_selection import train_test_split, cross_val_score 
from statistics import mean
from sklearn import preprocessing


import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split

 - Resources: https://github.com/NicolasHug/Surprise

In [5]:
df_total = pd.read_csv("beer_data/model_data.csv",index_col = 0)

In [6]:
df_total.head(10)

,Beer_id,Username_id,User Score
0,84627,30162,3.08
1,84627,11051,3.13
2,84627,24945,4.07
3,84627,30155,3.38
4,84627,30731,4.07
5,84627,1942,3.83
6,84627,21452,3.95
7,84627,25654,3.47
8,84627,3307,4.00
9,84627,8724,3.38


In [7]:
df_total.shape

(678042, 3)

In [8]:
df_total.columns = ["beer_id", "User_id", "ratings"]

In [9]:
df_total[df_total["User_id"] == 15186]

,beer_id,User_id,ratings
1607384,92551,15186,3.99


 - Converting the dataframe into the dataset required for susprise library

In [10]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import random

from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import GridSearchCV

In [11]:
reader = Reader(rating_scale=(0, 5,))

In [12]:
data = Dataset.load_from_df(df_total[['User_id', 'beer_id', 'ratings']], reader)

raw_ratings = data.raw_ratings

In [13]:
random.shuffle(raw_ratings)

In [14]:
threshold = int(.8 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]

In [15]:
data.raw_ratings = A_raw_ratings

 - Grid search to find the best number of epochs, regularization and factors
 ##### Takes a long time to run!

In [16]:
print('Grid Search...')
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005], 'n_factors' : [10,20,50]}
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
grid_search.fit(data)
algo = grid_search.best_estimator['rmse']
grid_search.best_estimator['rmse']





Grid Search...


In [17]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

from tabulate import tabulate
from six import iteritems

from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV

In [18]:
table = [[] for _ in range(len(grid_search.cv_results['params']))]
for i in range(len(grid_search.cv_results['params'])):
    for key in grid_search.cv_results.keys():
        table[i].append(grid_search.cv_results[key][i])

header = grid_search.cv_results.keys()
print(tabulate(table, header, tablefmt="rst"))

print()

for key, val in iteritems(grid_search.cv_results):
    print('{:<20}'.format("'" + key + "':"), end='')
    if isinstance(val[0], float):
        print([float('{:.2f}'.format(f)) for f in val])
    else:
        print(val)

==================  ==================  ==================  ================  ===============  ================  ===============  ==============  ================  ===============  ==================================================  ================  ==============  =================
  split0_test_rmse    split1_test_rmse    split2_test_rmse    mean_test_rmse    std_test_rmse    rank_test_rmse    mean_fit_time    std_fit_time    mean_test_time    std_test_time  params                                                param_n_epochs    param_lr_all    param_n_factors
==================  ==================  ==================  ================  ===============  ================  ===============  ==============  ================  ===============  ==================================================  ================  ==============  =================
          0.524614            0.522885            0.524236          0.523912      0.000742134                10          12.9716        1.91763  

In [19]:
print(grid_search.best_params['rmse'])


{'n_epochs': 10, 'lr_all': 0.005, 'n_factors': 10}


 - Training the best parameters 

In [20]:
trainset = data.build_full_trainset()
algo.fit(trainset)

In [21]:
predictions = algo.test(trainset.build_testset())
print('Biased accuracy on A,', end='   ')
accuracy.rmse(predictions)

Biased accuracy on A,   RMSE: 0.4374


0.4374473064007535

In [22]:
testset = data.construct_testset(B_raw_ratings)  # testset is now the set B
predictions = algo.test(testset)
print('Unbiased accuracy on B,', end=' ')
accuracy.rmse(predictions)

Unbiased accuracy on B, RMSE: 0.4682


0.4681965499068117

In [23]:
predictions

[Prediction(uid=24077, iid=35690, r_ui=4.83, est=3.952009373926649, details={'was_impossible': False}),
 Prediction(uid=9132, iid=22209, r_ui=3.06, est=3.2971696433806454, details={'was_impossible': False}),
 Prediction(uid=32604, iid=42005, r_ui=3.08, est=3.5194714549754904, details={'was_impossible': False}),
 Prediction(uid=3240, iid=36877, r_ui=4.47, est=4.2309794631111535, details={'was_impossible': False}),
 Prediction(uid=28428, iid=76847, r_ui=4.18, est=3.946301365796149, details={'was_impossible': False}),
 Prediction(uid=17338, iid=77260, r_ui=3.2, est=3.7697472897774467, details={'was_impossible': False}),
 Prediction(uid=29446, iid=88776, r_ui=3.59, est=3.5999366447332246, details={'was_impossible': False}),
 Prediction(uid=16188, iid=9427, r_ui=4.07, est=3.902906963667371, details={'was_impossible': False}),
 Prediction(uid=17576, iid=46978, r_ui=4.05, est=3.8497390961838454, details={'was_impossible': False}),
 Prediction(uid=13219, iid=78690, r_ui=3.53, est=3.64990032185

In [24]:
predictions

[Prediction(uid=24077, iid=35690, r_ui=4.83, est=3.952009373926649, details={'was_impossible': False}),
 Prediction(uid=9132, iid=22209, r_ui=3.06, est=3.2971696433806454, details={'was_impossible': False}),
 Prediction(uid=32604, iid=42005, r_ui=3.08, est=3.5194714549754904, details={'was_impossible': False}),
 Prediction(uid=3240, iid=36877, r_ui=4.47, est=4.2309794631111535, details={'was_impossible': False}),
 Prediction(uid=28428, iid=76847, r_ui=4.18, est=3.946301365796149, details={'was_impossible': False}),
 Prediction(uid=17338, iid=77260, r_ui=3.2, est=3.7697472897774467, details={'was_impossible': False}),
 Prediction(uid=29446, iid=88776, r_ui=3.59, est=3.5999366447332246, details={'was_impossible': False}),
 Prediction(uid=16188, iid=9427, r_ui=4.07, est=3.902906963667371, details={'was_impossible': False}),
 Prediction(uid=17576, iid=46978, r_ui=4.05, est=3.8497390961838454, details={'was_impossible': False}),
 Prediction(uid=13219, iid=78690, r_ui=3.53, est=3.64990032185

### RECOMMENDATIONS USING SURPRISE LIBRARY

In [25]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset



def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
        
    return top_n

In [26]:
top_n = get_top_n(predictions, n=10)


In [27]:
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

24077 [43523, 92294, 47768, 35690]
9132 [28811, 81966, 61882, 11122, 98717, 45971, 13006, 55131, 52818, 22209]
32604 [88860, 21166, 4802, 55516, 18991, 75415, 26682, 76721, 66452, 62417]
3240 [92954, 55845, 75291, 77440, 4642, 51018, 11189, 91744, 65032, 30589]
28428 [4443, 95032, 35584, 72221, 62345, 35444, 56446, 35559, 94259, 17307]
17338 [97188, 31042, 83768, 92337, 34413, 77260, 18353, 44852, 32131, 52638]
29446 [6448, 11476, 42632, 10243, 38745, 94981, 60624, 22044, 48500, 42963]
16188 [32915, 58012, 42699, 93571, 11868, 2194, 82333, 69101, 41263, 10989]
17576 [91743, 85252, 23338, 66452, 10738, 34460, 63768, 19498, 26517, 17027]
13219 [63772, 84525, 51890, 30744, 94783, 66505, 49713, 11241, 79511, 29857]
6882 [65552, 54146, 12322, 27863, 48496, 68894, 68282, 22027, 71689, 69812]
31141 [77909, 19485, 91418, 72871, 67925, 100183, 6128, 40607, 85694, 22024]
31255 [13819, 14660, 83527, 13703, 71221, 88797, 73306, 31852, 26297, 21356]
2459 [30252, 56573, 15716, 22358, 92651, 26499, 2

14334 [14555, 98201, 18146]
33828 [71986, 77903, 97103, 35258, 92096, 52825, 88089, 39775, 81243, 21210]
23664 [84566, 84630, 31755, 39146, 71706, 71824, 72568]
7576 [92965, 98922, 47579, 90339]
15339 [86386, 10700, 97319, 65763, 19719, 16903, 87996, 21001]
30474 [2405, 70537, 61201, 66920, 2406, 18733, 91839, 48735, 77251, 81785]
808 [42712, 14760, 25006, 74564, 19904, 53002, 33474, 56227, 54934, 58107]
8645 [26182, 55177, 33330, 55506, 65552, 76323, 23338, 47507, 44730, 14774]
30086 [42137, 84106, 67076, 10989, 94569, 68264, 97291, 11250, 11059, 15490]
19146 [19670, 11465, 75288, 27067, 6034, 23981, 4680, 31176, 44621, 38163]
6285 [92297, 49552, 49721]
19129 [83661, 85109, 4643, 83517, 83527, 78987, 26962, 22380, 36119, 32360]
18710 [97212, 46170]
13242 [77205, 99747, 33704, 51161, 51007, 40273]
10704 [70718, 63817, 10657, 23707, 35401, 13867, 39963, 55821, 81597, 42039]
10905 [11417, 89060, 1001, 19313, 45262, 35129, 55198, 85058, 86604, 7427]
24382 [5881]
7030 [54242, 24122, 43644,

23981 [75378, 4179, 62407, 84051, 5711, 69241, 54065, 80929, 4171, 28830]
27390 [36845]
1607 [27261, 8917, 47765, 17580, 93058]
7357 [46368, 81461, 94879, 41734, 50708, 13289]
21310 [83161, 100384, 69106, 70736, 30534, 63894, 51404, 69378, 58124, 42691]
2330 [23985, 12601, 77264, 12458, 62768, 79463, 83794, 12153, 24047, 34747]
17749 [73104, 59813, 59815, 28438, 40973, 37094, 42390, 37091, 54936, 19140]
14441 [59697]
221 [54242, 24821, 32184, 34566, 96837, 94064, 5475, 26523, 62384, 90671]
30423 [34460, 34564, 49881, 1604, 14175, 11807, 69791, 46983, 23664, 70272]
14366 [51825, 3536, 19125, 98537, 83386, 1786, 36367, 87321, 9748, 41451]
7449 [54065, 60411, 65194, 84588]
25224 [59291, 52636, 58107, 81749, 37957, 4489, 21383, 76441, 77633, 59073]
18877 [100270, 33452, 93628, 13721, 65032, 82718, 9702, 65316, 46260, 42347]
9822 [2293]
13853 [30677]
27505 [35013, 77273, 6303, 77265, 7928]
22672 [3522, 9038, 31147]
29367 [69228, 46778, 13316, 96555, 47817, 39621, 47901, 55490, 11020, 64819]

32638 [83247, 44537, 24410, 29131, 85309, 64404, 75579, 31412, 8488, 1335]
11935 [57877]
11888 [75291, 58540, 58541, 76323, 13676, 43366, 26513, 66309, 26596, 35968]
25575 [39743, 28234, 50576, 42784, 37171, 73145, 79686, 73230, 64575, 37359]
29099 [46272]
25068 [19852, 24136, 67792, 4699, 35722, 57235, 20508, 92727, 40236, 41947]
14152 [99191, 19485, 80483, 17374, 39879, 39913, 73831, 2977, 50227, 32186]
34367 [66343]
109 [27972, 27522, 92122, 6170, 48094, 14657, 81256, 24514, 7208, 4563]
14952 [7484, 3494, 83538, 11670]
19498 [9107, 94560]
10846 [55951, 61275, 72485, 62897, 18510, 75617, 32484, 18970, 85628, 78574]
28473 [78159]
19378 [54608, 83552, 6420, 48069, 71388, 59666, 37815, 74187, 89530]
28731 [39107]
17365 [62345, 82059, 28839, 9701, 65303, 19417, 24474, 93484, 61275, 86842]
30079 [46796, 46843, 26652, 82703]
1502 [45250, 16186, 28535, 16192, 100368]
7155 [5666, 25121, 17193, 90654, 16716, 77475, 77704, 2024, 63968, 53438]
4255 [11411, 35241]
7309 [76140, 76141]
468 [47604,

12010 [48289, 4205, 17364, 9978, 55888, 19493, 91546, 96751, 54651, 30512]
21099 [94281, 81093]
16023 [69698, 51744, 81741, 1818, 44982, 35143, 62076, 60335, 14558, 81237]
8330 [9989, 68474, 84523]
18634 [98663]
26949 [62417, 92805, 48620, 39966]
28089 [11363]
8319 [97643]
31357 [78727, 2034, 65247, 94752, 56918, 78004, 8013, 5988]
16534 [50006, 58273, 89424, 82358, 2069, 5182, 96167, 30925]
35828 [44748, 16448, 81603, 40100, 8417, 38120, 83447, 44492, 68103, 77657]
26087 [32444, 9057]
28711 [75017, 9875, 60623, 76766, 81617, 58403, 67413, 44073]
3688 [25952]
24708 [94177, 59475, 28327, 71900, 91336, 9904, 88009, 28971, 91666]
21010 [13703, 84160, 2013, 96763]
2870 [21464]
24014 [47503, 71540]
4168 [60722, 60724]
24810 [97614]
30071 [1604, 51674, 92391, 75040, 59896, 53379, 73440, 28327, 90345, 25243]
19624 [45251, 91188, 63982, 52904, 11268, 10194, 14341, 18593, 98622, 26925]
5644 [27484]
32408 [71865]
13193 [44026]
10636 [28439, 2903]
6689 [35038, 54525]
6532 [58808, 89030]
26385 [75

...

4266 [46934]
17785 [23786]
12426 [73174]
6856 [32999, 41122]
5942 [51266]
10813 [51551]
15970 [67570]
3860 [82597]
16824 [68271]
8804 [79201]
32900 [53429, 95443]
27380 [11346]
27130 [58236, 30954]
4058 [53273]
26203 [19481, 36041, 38889]
1861 [10623]
31778 [92031, 84092, 88431]
30797 [10897]
11528 [82135, 58630]
36005 [54020]
25724 [56017]
13785 [15759]
16743 [6672]
4827 [84626]
29643 [47445, 87018]
13925 [56222, 56226, 92956]
30232 [42691, 96815, 52010, 28277, 40994]
5674 [5013]
34265 [89186, 18167]
6488 [14617, 588, 19949]
27004 [68597]
30971 [17771]
28164 [536]
14914 [83629, 40184]
27184 [22666, 91292]
25967 [85014]
29115 [51671, 36101, 11805, 71387, 395]
19658 [13316, 70500]
30393 [15004]
34940 [66268]
5948 [16834]
10894 [29628]
27830 [75036]
33347 [65440]
10712 [14756]
34707 [42762, 21199]
12764 [45791]
4816 [32166]
6956 [9577]
14933 [65732]
26344 [94171]
3185 [90679]
7702 [67888]
29060 [60919]
7841 [36884]
7144 [54119]
14639 [86224]
12791 [9745, 32299]
4232 [6313]
28915 [75161]


32449 [33744, 35662]
1144 [9978]
16477 [43175]
16755 [82115, 98593]
35760 [29314]
3739 [18513]
24273 [58012]
32031 [27377, 97924]
36089 [37569]
5884 [41550]
27499 [32611]
6492 [87913]
34593 [86461]
24558 [64246, 99777]
26235 [41346]
30877 [22400]
13820 [20985, 64467]
33406 [39018]
11379 [56367]
6437 [68285, 40943]
20778 [38627]
1616 [40880]
3399 [52791]
16890 [44067]
20402 [26736]
33476 [87887, 48197]
5624 [35409]
19248 [2087]
34678 [36628]
9552 [21074]
4531 [7492]
29114 [14586]
9534 [28374]
6052 [98677]
23177 [98121]
26361 [30471]
6268 [92434]
1443 [24008, 16894]
26013 [69922, 14519]
35787 [63830]
35614 [72643, 94325]
13997 [90978]
24995 [23853, 6394]
31407 [66525]
3445 [6684]
30655 [52187]
23809 [9293, 88899, 62920]
6517 [41204, 23811]
34509 [65358]
31801 [57740]
11759 [34013]
5523 [58526, 73803, 46226]
24761 [63760]
25770 [47012]
10605 [25231, 80381]
10549 [42935]
16655 [28120, 78260, 21453, 91404]
12974 [32242]
35921 [95278]
29482 [64111]
20832 [74120]
28282 [24756]
10044 [95460]
3

1229 [37097]
33695 [36457]
7047 [67371]
4407 [56203]
18489 [35414]
8305 [57526]
584 [69393, 11110]
6823 [50841]
16828 [50836]
17315 [76414]
948 [47509]
9429 [96365]
8399 [50552]
1744 [21826]
27939 [2741]
16434 [88603]
5777 [20132]
23984 [46289]
34085 [77681]
12103 [25230]
21849 [45275]
3417 [66309]
12545 [40607]
33388 [36292]
29371 [9822]
14845 [46794]
3676 [54331]
15770 [33486]
12257 [97131]
7187 [39427]
19998 [27396]
7022 [77354]
22758 [63788]
21559 [64049]
26252 [49170, 95311]
7454 [90167]
1057 [30085, 86366, 90068]
25637 [52048]
25070 [38576]
11754 [52244]
29959 [38996]
3622 [9851]
8910 [8650]
2510 [58256, 17140]
26488 [22020]
20150 [34707]
27200 [36332]
36077 [88490]
29559 [40515]
28879 [70506, 83097]
2503 [42718, 48819]
15060 [18854]
9526 [7668]
15367 [33670]
14312 [90336]
35256 [99881]
17244 [6361]
28677 [99842]
33110 [69842, 24482]
10347 [85500]
18090 [13084]
21044 [68438]
11254 [49091]
7984 [15988]
5811 [47566]
18259 [9923]
32535 [78431]
13598 [30096]
23897 [85961]
11142 [8747

26685 [24552]
34545 [65198]
30624 [32528]
17556 [87503]
11784 [21495]
13864 [1161]
8458 [42746]
27204 [43706]
10463 [97162]
1327 [53192]
23262 [25732, 36257]
30264 [22907]
31575 [5709]
19945 [50458]
25587 [49959]
15555 [35309]
7857 [13589]
11113 [77220]
2101 [62885]
27688 [69506]
3155 [48636]
33475 [29091]
32515 [53874]
34725 [50236]
26823 [42343]
11812 [68282]
4810 [95892]
672 [36853]
13959 [73075]
34229 [25121]
33912 [46414]
30580 [50414]
31857 [63142]
14395 [11243]
31537 [60993, 69302]
18215 [39208]
8900 [21076]
18146 [77667]
657 [19147]
11810 [70087]
12467 [75637]
19236 [36615]
17930 [62309]
7939 [17696]
33448 [59740]
22904 [77287]
7191 [69224]
18985 [95770]
24846 [20252]
11157 [74014]
22592 [40397]
4156 [29400]
30965 [12044]
4343 [73057]
9600 [89915]
34616 [80739]
23927 [93212]
27369 [70409]
405 [91349]
29601 [73297]
11970 [8642]
12588 [62664]
15294 [10487]
35874 [19039]
19595 [82380]
33371 [82673]
32306 [26725]
3436 [13912]
29889 [85141]
4042 [52262]
28375 [25654]
19853 [23479]
3

9857 [99179]
35824 [39567]
277 [83930]
9300 [18623]
10232 [66378]
13524 [72415]
7748 [26028]
5766 [24266, 64553, 95860]
23053 [17096]
14712 [12720]
7619 [35347]
1729 [36727]
1263 [45047]
24797 [35382]
32177 [39993]
11823 [98866]
26227 [65813]
34480 [68908]
14584 [67583]
23588 [99271]
26283 [63927]
29529 [20414]
987 [94803]
24927 [89436]
32856 [97228]
18726 [69757]
22110 [67637]
20831 [18991]
13385 [37550]
14979 [100052, 98442]
3710 [38485]
2547 [22475]
30599 [28325]
3677 [63399]
8062 [14139]
21482 [7902]
330 [1580]
18113 [62539]
21116 [35386]
17582 [63873]
31158 [51056]
6143 [66629]
5693 [80854]
26272 [12473]
249 [77937]
35009 [83510]
6680 [41485]
25842 [13674, 52731]
30582 [43442]
21974 [29264]
4268 [19673, 42720]
26458 [28101]
1096 [99502]
6157 [96762]
35531 [78805, 11608]
7818 [23852]
1936 [63514]
11248 [42025]
31641 [36770]
8911 [86358]
22439 [13056]
10249 [53543, 100321]
26812 [30435]
8546 [48211]
33994 [78327]
19960 [69111, 66953]
25505 [85170]
8283 [15955]
21437 [59348, 37095]
3

In [28]:
df_total[df_total["User_id"] == 15186].sort_values(by="ratings", ascending=False)

,beer_id,User_id,ratings
1607384,92551,15186,3.99


In [29]:
df_total

,beer_id,User_id,ratings
0,84627,30162,3.08
1,84627,11051,3.13
2,84627,24945,4.07
3,84627,30155,3.38
4,84627,30731,4.07
...,...,...,...
2253448,31152,21876,3.74
2253537,92364,7908,4.14
2253959,51824,7292,4.47
2254038,51824,8371,3.58


In [30]:
uid = str(800000)
iid = str(80468)
pred = algo.predict(uid, iid, verbose=True)

user: 800000     item: 80468      r_ui = None   est = 3.78   {'was_impossible': False}


In [43]:
top_n

defaultdict(list,
            {24077: [(43523, 4.206186835091882),
              (92294, 4.135262202196569),
              (47768, 4.022541994523361),
              (35690, 3.952009373926649)],
             9132: [(28811, 3.4364924840198174),
              (81966, 3.413739939584655),
              (61882, 3.3791307631725274),
              (11122, 3.3671698989766568),
              (98717, 3.3375148019137293),
              (45971, 3.3182412797171983),
              (13006, 3.31689212506297),
              (55131, 3.3057548974297633),
              (52818, 3.2983974534252782),
              (22209, 3.2971696433806454)],
             32604: [(88860, 3.9386633504369537),
              (21166, 3.9019907190883107),
              (4802, 3.8737628645533713),
              (55516, 3.8643384578628863),
              (18991, 3.864143692015404),
              (75415, 3.8366027180720144),
              (26682, 3.836555356425562),
              (76721, 3.82745148232289),
              (66452, 3.82

In [48]:
for key in top_n:
   print("user: %s , beer_recommendation: %s" % (key, top_n[key]))

user: 24077 , beer_recommendation: [(43523, 4.206186835091882), (92294, 4.135262202196569), (47768, 4.022541994523361), (35690, 3.952009373926649)]
user: 9132 , beer_recommendation: [(28811, 3.4364924840198174), (81966, 3.413739939584655), (61882, 3.3791307631725274), (11122, 3.3671698989766568), (98717, 3.3375148019137293), (45971, 3.3182412797171983), (13006, 3.31689212506297), (55131, 3.3057548974297633), (52818, 3.2983974534252782), (22209, 3.2971696433806454)]
user: 32604 , beer_recommendation: [(88860, 3.9386633504369537), (21166, 3.9019907190883107), (4802, 3.8737628645533713), (55516, 3.8643384578628863), (18991, 3.864143692015404), (75415, 3.8366027180720144), (26682, 3.836555356425562), (76721, 3.82745148232289), (66452, 3.8259685749923693), (62417, 3.825380512449632)]
user: 3240 , beer_recommendation: [(92954, 4.355731691880113), (55845, 4.350093678204478), (75291, 4.327021989084006), (77440, 4.319481307961794), (4642, 4.319476274744851), (51018, 4.299019272710293), (11189, 

user: 25763 , beer_recommendation: [(5722, 4.026409150732137), (29845, 3.812677125679991), (65801, 3.7796747334446787), (47102, 3.765879170106408), (1117, 3.6941444147008715), (69878, 3.680860784948294), (73506, 3.6799251299458966), (54571, 3.6778276504251637), (74299, 3.6315975536280907), (59341, 3.601748333154912)]
user: 5790 , beer_recommendation: [(21166, 4.160657344208448), (76072, 4.04274486496722), (92237, 3.740292496625096), (31163, 3.7348854143562953), (68305, 3.6833347085300474), (63739, 3.6693525918070704), (85866, 3.625102416342104), (88627, 3.509927279749805), (75213, 3.381315716025928)]
user: 19829 , beer_recommendation: [(25297, 3.7357040628085514), (83443, 3.606266633624049), (67026, 3.5978924491625173), (91044, 3.551585876232704), (25304, 3.5496095678059834), (25858, 3.471200027327022), (73417, 3.4253220543040372), (87458, 3.403891031946848), (67556, 3.399733256160186), (53733, 3.010709283605515)]
user: 31378 , beer_recommendation: [(73306, 3.8721792032833737), (77713,

user: 35609 , beer_recommendation: [(42980, 3.8756803805605187), (35459, 3.8078746970958273), (26755, 3.76071602441308), (8897, 3.734468447837665), (84994, 3.66773772853997), (98120, 3.551634073224938), (56722, 3.546588395945519), (37949, 3.4420339893131704), (81040, 3.423508980792012), (97723, 3.163647875194661)]
user: 30878 , beer_recommendation: [(7105, 3.730171541513307), (7087, 3.680138879082677), (46421, 3.6008502663093616), (30116, 3.5460456647970546), (53556, 3.3992161994851755), (84162, 3.2021271811220853)]
user: 4641 , beer_recommendation: [(68933, 3.9369405505646675), (4234, 3.9033144309619305), (9769, 3.8405361970492136), (50183, 3.7539326754530635), (17325, 3.749573852735303), (86019, 3.740707843629267), (85494, 3.7327901101976835), (90567, 3.7289340234849018), (8762, 3.7199685401955347), (28367, 3.70700342651288)]
user: 9814 , beer_recommendation: [(8838, 3.5822220955185973), (45634, 3.5353983950337016), (65784, 3.151114401352928)]
user: 8132 , beer_recommendation: [(4558

user: 5472 , beer_recommendation: [(64240, 3.924868144219949), (39396, 3.802973612002483), (21647, 3.788686252687908), (82885, 3.737454581371257), (38796, 3.6915634527682366), (25872, 3.688323405226366), (94855, 3.550089323936975)]
user: 20741 , beer_recommendation: [(83478, 3.5520264601987592), (58918, 3.492006405589767), (65019, 3.4149195226990825)]
user: 16660 , beer_recommendation: [(37975, 4.260114169566355), (13619, 4.1806507988465205), (46764, 4.1786649951819586), (92263, 4.123247735048186), (21147, 4.120335276787644), (24062, 4.117168621642616), (51232, 4.106137643010415), (81785, 4.07410323393279), (43335, 4.066277525337589), (36870, 4.060349431241054)]
user: 18202 , beer_recommendation: [(95520, 3.749982002628535)]
user: 26366 , beer_recommendation: [(76797, 4.038199436776135), (28237, 3.9113095885382907), (32430, 3.9000285899469613), (76967, 3.806596469707538), (65412, 3.8052874956336513), (11727, 3.79805562995009), (61832, 3.784405846579276), (18652, 3.784387472954795), (51

user: 24938 , beer_recommendation: [(78080, 3.624110644511766)]
user: 16473 , beer_recommendation: [(28385, 4.45371132698642), (10199, 4.4379655549796295), (83498, 4.42695231424289), (27363, 4.382517675406543), (67352, 4.372089655308719), (56624, 4.363244164233499), (67074, 4.359027980996545), (89389, 4.308535222696274), (7535, 4.292684254325001), (43931, 4.292227589115183)]
user: 16683 , beer_recommendation: [(86644, 4.156671346688674), (26349, 4.0642066971468935), (5955, 4.057805559031175), (526, 3.8667549039888414)]
user: 8775 , beer_recommendation: [(93640, 3.7357847063701546), (4520, 3.725898934289501), (65923, 3.717108013679699), (7614, 3.7150776087383286), (95241, 3.6336162575918785)]
user: 6426 , beer_recommendation: [(65784, 3.0735359328023075)]
user: 23087 , beer_recommendation: [(52937, 3.7502248358406347)]
user: 4047 , beer_recommendation: [(37422, 4.207617362435609), (80606, 4.165995028596673), (53531, 4.134056603415601), (47751, 4.074933084843314), (18830, 4.0470299070474

...

user: 35800 , beer_recommendation: [(35150, 3.7985165666672205)]
user: 16480 , beer_recommendation: [(7032, 3.857428708540537)]
user: 34519 , beer_recommendation: [(12446, 3.708189011083988)]
user: 9107 , beer_recommendation: [(33295, 3.7762935883325692)]
user: 22356 , beer_recommendation: [(57312, 3.849881320537063)]
user: 5392 , beer_recommendation: [(75298, 3.758049599307712)]
user: 4975 , beer_recommendation: [(59204, 3.7585290520793926)]
user: 33596 , beer_recommendation: [(71509, 3.7910339324504454)]
user: 22834 , beer_recommendation: [(75146, 3.84138670336027)]
user: 30358 , beer_recommendation: [(71015, 4.0816444418056665), (14994, 4.010147908259512), (51382, 3.8856810617542967)]
user: 17772 , beer_recommendation: [(70736, 3.982629109656802)]
user: 10987 , beer_recommendation: [(63068, 3.799484640005307)]
user: 22275 , beer_recommendation: [(22656, 3.8604407024125873)]
user: 6069 , beer_recommendation: [(68258, 3.8106645267855477)]
user: 23516 , beer_recommendation: [(69983, 3.

user: 31496 , beer_recommendation: [(27020, 3.8798642602240405)]
user: 17404 , beer_recommendation: [(94791, 3.8562168766650426), (61011, 3.7168562195568358)]
user: 3592 , beer_recommendation: [(37513, 3.8409466401174113), (22122, 3.546609442505905)]
user: 1817 , beer_recommendation: [(91888, 3.8207304205217074)]
user: 26417 , beer_recommendation: [(79743, 3.6489218613701286)]
user: 1530 , beer_recommendation: [(77741, 3.8206401465932713)]
user: 10028 , beer_recommendation: [(49161, 3.812449708342424)]
user: 14379 , beer_recommendation: [(34143, 3.682111421002411)]
user: 6092 , beer_recommendation: [(26600, 4.010938580482444)]
user: 9514 , beer_recommendation: [(6988, 3.622307923972903)]
user: 31114 , beer_recommendation: [(54982, 3.794385462781967)]
user: 15189 , beer_recommendation: [(34293, 3.9398462537502903)]
user: 35876 , beer_recommendation: [(84504, 4.211232261565521)]
user: 16414 , beer_recommendation: [(22176, 3.030247954060158)]
user: 11773 , beer_recommendation: [(27033, 3.

user: 5877 , beer_recommendation: [(27668, 3.656274240601431)]
user: 11253 , beer_recommendation: [(46472, 3.831099614593906)]
user: 20885 , beer_recommendation: [(77287, 3.739637412359611)]
user: 4349 , beer_recommendation: [(85495, 3.6495957012777094)]
user: 28226 , beer_recommendation: [(83878, 3.7238912574962546)]
user: 18231 , beer_recommendation: [(863, 3.7762935883325692)]
user: 26685 , beer_recommendation: [(24552, 3.8404646514866854)]
user: 34545 , beer_recommendation: [(65198, 3.7801970356686176)]
user: 30624 , beer_recommendation: [(32528, 3.640341659075559)]
user: 17556 , beer_recommendation: [(87503, 3.903793926776702)]
user: 11784 , beer_recommendation: [(21495, 3.789126007267918)]
user: 13864 , beer_recommendation: [(1161, 3.5557836055707965)]
user: 8458 , beer_recommendation: [(42746, 3.8755374791553794)]
user: 27204 , beer_recommendation: [(43706, 3.3634248123761674)]
user: 10463 , beer_recommendation: [(97162, 3.6965838361188648)]
user: 1327 , beer_recommendation: [(5

user: 7244 , beer_recommendation: [(21471, 3.8412860793793895)]
user: 1009 , beer_recommendation: [(23407, 3.5191088827977075)]
user: 15412 , beer_recommendation: [(27940, 3.4371509869887396)]
user: 10057 , beer_recommendation: [(75107, 3.129536506668496)]
user: 33881 , beer_recommendation: [(85987, 3.812033158726272)]
user: 5094 , beer_recommendation: [(92773, 4.039459983776002)]
user: 34354 , beer_recommendation: [(76200, 3.8703246870594596)]
user: 28173 , beer_recommendation: [(3058, 3.6953670716033433)]
user: 12828 , beer_recommendation: [(19373, 3.58333559211002)]
user: 161 , beer_recommendation: [(21209, 3.810291475781588)]
user: 23566 , beer_recommendation: [(38414, 3.091316334162947)]
user: 32793 , beer_recommendation: [(89444, 3.797225645721158)]
user: 33579 , beer_recommendation: [(15939, 3.9126671198726557)]
user: 15590 , beer_recommendation: [(26431, 3.9653579254864746)]
user: 17797 , beer_recommendation: [(12424, 3.2235152368597007)]
user: 32288 , beer_recommendation: [(58

user: 20886 , beer_recommendation: [(52292, 3.67147405123373)]
user: 21286 , beer_recommendation: [(57610, 3.7588548761921183)]
user: 19176 , beer_recommendation: [(96149, 3.7893720901061316)]
user: 12774 , beer_recommendation: [(15612, 3.6212155369843106)]
user: 18563 , beer_recommendation: [(47041, 3.7853911858135)]
user: 21833 , beer_recommendation: [(62734, 3.6351361430547073)]
user: 15814 , beer_recommendation: [(42996, 3.8869397436834427)]
user: 15055 , beer_recommendation: [(49382, 3.319010370358495)]
user: 21727 , beer_recommendation: [(66719, 3.5850835888974424)]
user: 10178 , beer_recommendation: [(32419, 3.4962973473150236)]
user: 18559 , beer_recommendation: [(81537, 3.841059188671332)]
user: 16628 , beer_recommendation: [(97604, 3.6316917139718665)]
user: 14729 , beer_recommendation: [(93155, 3.7352364386356145)]
user: 17435 , beer_recommendation: [(27179, 3.7703515578760394)]
user: 12786 , beer_recommendation: [(19372, 3.4871310054464826)]
user: 7226 , beer_recommendation

In [46]:
# Recommendation user 4579 - same as used as example for neural network model
top_n[4579]

[(11422, 3.9007467388040196)]

In [49]:
top_n[24077]

[(43523, 4.206186835091882),
 (92294, 4.135262202196569),
 (47768, 4.022541994523361),
 (35690, 3.952009373926649)]